#### Calculando Distância e Duração de Rotas Utilizando Diferentes Meios de Transporte
Calculating Distance and Duration of Routes Using Different Means of Transportation

In [33]:
# Importando Bibliotecas | Importing Libraries

import requests
import time
import json
import folium
from fpdf import FPDF
import os
from dotenv import load_dotenv
import io
from PIL import Image

In [34]:
# Chave da API | API Key
load_dotenv() # Lê e carrega variáveis de .env | # Reads and loads variables from .env
api_key = os.getenv('ors_api_key')
if not api_key:
    raise ValueError("Chave de API não encontrada | API key not found")

# Para obter sua chave, visite https://account.heigit.org/login e crie uma conta. 
# Assim que sua conta for ativada, vá para o seu Painel (Dashboard), copie a longa sequência de caracteres que aparece em Chave Básica (Basic Key) e a atribua a variável ors_api_key.

# To get your key, visit https://account.heigit.org/login and create an account.
# Once your account is activated, go to your Dashboard, copy the long string that appears under Basic Key, and assign it to the ors_api_key variable.

In [35]:
# Configuração dos Diretórios | Setting up Directories
output_base_dir = os.path.join('..','outputs') 
figures_dir = os.path.join(output_base_dir, "figures")
results_dir = os.path.join(output_base_dir, "results")

# Verifica se os diretórios existem e os cria se não existirem
# Checks if the directories exist and creates them if they don't
os.makedirs(figures_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

In [36]:
# Origem e Destinos | Source and Destinations
source = (-22.890656, -43.123864) # Caminho Niemeyer, Niterói (RJ)

# Destinos | Destination
destination_cities = {
    'Rio de Janeiro': (-22.9519, -43.2105), # Cristo Redentor, Rio de Janeiro (RJ)
    'Petropolis': (-22.5050, -43.1812), # Palácio de Cristal, Petrópolis (RJ)
    'Paraty': (-23.2195, -44.7170) # Casa da Cultura de Paraty, Paraty (RJ)
}

In [37]:
# Função para Calcular Distância e Duração | Function to Calculate Distance and Duration

def get_route_info(source_coords, dest_coords, profile, api_key):
    """
    Calcula a distância e a duração de uma rota entre dois pontos usando a API OpenRouteService.
    Calculates the distance and duration of a route between two points using the OpenRouteService API.
    """
    params = {
        'api_key': api_key,
        'start': '{},{}'.format(source_coords[1], source_coords[0]),
        'end': '{},{}'.format(dest_coords[1], dest_coords[0])
    }
    url = f"https://api.openrouteservice.org/v2/directions/{profile}"
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        summary = data['features'][0]['properties']['summary']
        geometry = data['features'][0]['geometry']['coordinates']

        distance_km = summary.get('distance', 0) / 1000
        duration_seconds = summary.get('duration', 0)
        
        hours = int(duration_seconds // 3600)
        minutes = int((duration_seconds % 3600) // 60)
        seconds = int(duration_seconds % 60)
        duration_formatted_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        
        return distance_km, duration_formatted_str, geometry
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para o perfil '{profile}': {e}")
        if response is not None:
            print(f"Resposta da API: {response.text}")
        return None, None, None
    except (KeyError, IndexError):
        print(f"Erro ao analisar dados para o perfil '{profile}'. Resposta: {response.text}")
        return None, None, None

In [38]:
# Função para Gerar e Salvar Mapas | Function to Generate and Save Maps

def plot_route_on_map(source_coords, dest_coords, route_geometry, profile_name, city_name, folder_path=figures_dir):
    """
    Cria um mapa com a rota e o salva como um arquivo PNG.
    Create a map with the route and save it as a PNG file.
    Esta versão usa o método interno do Folium e a biblioteca Pillow, sem a necessidade de wkhtmltopdf.
    This version uses Folium's internal method and the Pillow library, without the need for wkhtmltopdf.
    """
    # Inverter as coordenadas para o formato esperado pelo Folium (lat, lon)
    # Invert the coordinates to the format expected by Folium (lat, lon)
    geometry_folium = [list(reversed(coords)) for coords in route_geometry]

    # Criar o mapa centralizado no ponto de origem
    # Create the map centered on the origin point
    m = folium.Map(location=[source_coords[0], source_coords[1]], zoom_start=8)
    folium.PolyLine(locations=geometry_folium, color='blue', weight=5, opacity=0.7).add_to(m)
    folium.Marker(location=source_coords, popup=f"Origem: Niterói", icon=folium.Icon(color='green', icon='play', prefix='fa')).add_to(m)
    folium.Marker(location=dest_coords, popup=f"Destino: {city_name}", icon=folium.Icon(color='red', icon='stop', prefix='fa')).add_to(m)

    # Salvar o mapa como um arquivo HTML temporário
    # Save the map as a temporary HTML file
    map_html_path = os.path.join(folder_path, f"mapa_temp_{profile_name}_{city_name}.html")
    m.save(map_html_path)

    map_png_path = os.path.join(folder_path, f"mapa_{profile_name}_{city_name}.png")

    try:
        # Lê o HTML do mapa e converte para um objeto de imagem usando o método interno do Folium
        # Read the map HTML and convert it to an image object using Folium's internal method
        img_data = m._to_png(5) # O número 5 é o tempo de espera em segundos para o render
        img = Image.open(io.BytesIO(img_data))
        img.save(map_png_path)
        
        # Remove o arquivo HTML temporário
        # Remove the temporary HTML file
        os.remove(map_html_path)

        if os.path.exists(map_png_path):
            print(f"Imagem PNG salva com sucesso em {map_png_path}")
            return map_png_path
        else:
            print(f"Erro: O arquivo de imagem não foi encontrado em {map_png_path}")
            return None
            
    except Exception as e:
        print(f"Erro ao gerar imagem: {e}")
        return None

In [39]:
# Função para Gerar Relatório em PDF | Function to Generate PDF Report

def generate_pdf_report(report_data, pdf_filename="relatorio_duracao_distancia_rotas.pdf", folder_path=results_dir):
    """
    Gera um relatório em PDF com informações de rotas e as imagens dos mapas.
    Generates a PDF report with route information and map images.
    """
    pdf_output_path = os.path.join(folder_path, pdf_filename)
    
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt="Relatório de Duração e Distância de Rotas", ln=True, align='C')
    pdf.ln(10)
    
    # Adiciona conteúdo para cada rota
    # Add content for each route
    for item in report_data:
        # Título do meio de transporte
        # Title of the mode of transport
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(200, 10, txt=f"Meio de Transporte: {item['profile_name_pt'].capitalize()} ({item['profile_key']})", ln=True, align='L')
        pdf.ln(2)
        
        # Título da rota
        # Title of the route
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(200, 10, txt=f"Rota de Niterói para {item['city_name']}", ln=True, align='L')
        
        # Adiciona a distância e duração
        # Add distance and duration
        pdf.set_font("Arial", size=10)
        pdf.cell(200, 5, txt=f"Distância: {item['distance_km']:.2f} km", ln=True, align='L')
        pdf.cell(200, 5, txt=f"Duração: {item['duration_formatted_str']} h", ln=True, align='L')
        pdf.ln(5)
        
        # Adiciona a imagem
        # Add the image
        try:
            if 'img_path' in item and item['img_path'] and os.path.exists(item['img_path']):
                pdf.image(item['img_path'], w=180)
            else:
                pdf.multi_cell(0, 5, "Imagem não gerada ou não encontrada para esta rota.")
            pdf.ln(10)
        except Exception as e:
            print(f"Não foi possível adicionar a imagem ao PDF para a rota {item['city_name']} ({item['profile_name_pt']}): {e}")
            
    pdf.output(pdf_output_path)
    print(f"\nRelatório em PDF gerado com sucesso: {pdf_output_path}")

In [40]:
# Execução das Funções | Execution of Functions

print("--- Calculando rotas para todos os destinos ---")

# Nova estrutura de dados para o relatório, agora uma lista de dicionários
# New data structure for the report, now a list of dictionaries
report_data = []

niteroi_coords = source

means_of_transportation = {
    'driving-car': 'carro',
    'foot-walking': 'caminhada',
    'cycling-regular': 'bicicleta regular'
}

for profile_key, profile_name_pt in means_of_transportation.items():
    for city_name, coords in destination_cities.items():
        print(f"  Calculando rota de Niterói para {city_name} com o perfil '{profile_name_pt}'...")
        
        distance_km, duration_formatted_str, geometry = get_route_info(niteroi_coords, coords, profile_key, api_key)
        
        if distance_km is not None:
            print(f"    Distância: {distance_km:.2f} km")
            print(f"    Duração: {duration_formatted_str} h")
            
            img_path = plot_route_on_map(niteroi_coords, coords, geometry, profile_key, city_name)
            if img_path:
                # Adiciona um dicionário com todos os dados da rota à lista
                # Add a dictionary with all the route data to the list
                report_data.append({
                    'profile_key': profile_key,
                    'profile_name_pt': profile_name_pt,
                    'city_name': city_name,
                    'distance_km': distance_km,
                    'duration_formatted_str': duration_formatted_str,
                    'img_path': img_path
                })
        else:
            print(f"    Não foi possível obter a rota para {city_name} com o perfil {profile_key}.")
        
        time.sleep(2)
        print("  " + "-" * 25)

print("\n--- Finalizado | Finish ---")

# Chama a função para gerar o relatório PDF no final, passando a nova estrutura de dados
# Call the function to generate the PDF report at the end, passing the new data structure
generate_pdf_report(report_data)

--- Calculando rotas para todos os destinos ---
  Calculando rota de Niterói para Rio de Janeiro com o perfil 'carro'...
    Distância: 33.78 km
    Duração: 00:49:01 h
Imagem PNG salva com sucesso em ..\outputs\figures\mapa_driving-car_Rio de Janeiro.png
  -------------------------
  Calculando rota de Niterói para Petropolis com o perfil 'carro'...
    Distância: 75.62 km
    Duração: 01:10:23 h
Imagem PNG salva com sucesso em ..\outputs\figures\mapa_driving-car_Petropolis.png
  -------------------------
  Calculando rota de Niterói para Paraty com o perfil 'carro'...
    Distância: 256.53 km
    Duração: 03:54:40 h
Imagem PNG salva com sucesso em ..\outputs\figures\mapa_driving-car_Paraty.png
  -------------------------
  Calculando rota de Niterói para Rio de Janeiro com o perfil 'caminhada'...
    Distância: 16.85 km
    Duração: 02:52:53 h
Imagem PNG salva com sucesso em ..\outputs\figures\mapa_foot-walking_Rio de Janeiro.png
  -------------------------
  Calculando rota de Niter